In [1]:
# Version 3.0.0
# Title: Flow Distribution in a Doublet EGS
# Authors: Pranay Asai (UoU) & Robert Podgorney (INL)
# Edited by: Pranay Asai
# Date: 06/26/2021
# Updates:
#     1. Added Version History and Updates
#     2. Addition of Adaptive Perforations
#     3. High language variables
#     4. Code Cleanup

In [2]:
import numpy as np  #Importing NumPy
import math 

In [3]:
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

In [4]:
#Calculate fracture permeabilities
import random
def FracturePermeability(n,Kb,FractureWidth,FractureHeight,premact):
    Kf=np.zeros(n)
    for i in range(n):
        if premact==0:
            Kf[i]=Kb
            #Af[i]=FractureWidth*FractureHeight
        else:
            rndnum=(random.uniform(1, 10))
            rndnum1=(random.choice([-1, 0, 1]))  #Magnitude
            #Wf[i]=rndnum*(10**rndnum1)*(12*Kb)**0.5
            #Kf[i]=(Wf[i]**2)/12
            Kf[i]=rndnum*Kb*(10**rndnum1)
            #Af[i]=Wpseudo*breadth
    return Kf

In [5]:
#Adaptive Perforations
def AdaptivePerforation(WellsOrientation,VariablePermeability,AdaptivePerf,NumberOfFractures,Diameter_Perforation,BaseDiameter_Perforation,Permeability_Fracture):
    GMean = math.prod(Permeability_Fracture)**(1/len(Permeability_Fracture))
    if WellsOrientation==1 or WellsOrientation==3:
        for i in range(NumberOfFractures):
            if VariablePermeability==0:
                if AdaptivePerf==1:
                    #Diameter_Perforation[i]=BaseDiameter_Perforation*(1.3**(math.log(GMean/Permeability_Fracture[i],10)))
                    Diameter_Perforation[i]=((i+2)**0.4/NumberOfFractures**0.3)*BaseDiameter_Perforation
                else:
                    Diameter_Perforation[i]=BaseDiameter_Perforation
            else:
                if AdaptivePerf==1:
                    Diameter_Perforation[i]=BaseDiameter_Perforation*(1.3**(math.log(GMean/Permeability_Fracture[i],10)))
                else:
                    Diameter_Perforation[i]=BaseDiameter_Perforation
    elif WellsOrientation==2:
        if NumberOfFractures%2==0:
            for i in range(int(NumberOfFractures/2)):
                if VariablePermeability==0:
                    if AdaptivePerf==1:
                        Diameter_Perforation[i]=((i+2)**0.4/NumberOfFractures**0.3)*BaseDiameter_Perforation
                    else:
                        Diameter_Perforation[i]=BaseDiameter_Perforation
                    Diameter_Perforation[int(NumberOfFractures-1)-i]=Diameter_Perforation[i]
                else:
                    if AdaptivePerf==1:
                        Diameter_Perforation[i]=BaseDiameter_Perforation*(1.3**(math.log(GMean/Permeability_Fracture[i],10)))
                    else:
                        Diameter_Perforation[i]=BaseDiameter_Perforation        
        else:
            for i in range(int((NumberOfFractures+1)/2)):
                if VariablePermeability==0:
                    if AdaptivePerf==1:
                        Diameter_Perforation[i]=((i+2)**0.4/NumberOfFractures**0.3)*BaseDiameter_Perforation
                    else:
                        Diameter_Perforation[i]=BaseDiameter_Perforation
                    if i!=int((NumberOfFractures-1)/2):
                        Diameter_Perforation[int(NumberOfFractures-1)-i]=Diameter_Perforation[i]
                else:
                    if AdaptivePerf==1:
                        Diameter_Perforation[i]=BaseDiameter_Perforation*(1.3**(math.log(GMean/Permeability_Fracture[i],10)))
                    else:
                        Diameter_Perforation[i]=BaseDiameter_Perforation
    return Diameter_Perforation

In [6]:
#Reynolds Number
def Rep(q,D):
    re=1000*(q/(math.pi*D**2*0.25))*D/(8.91e-4)
    return re

In [7]:
#Colebrook–White equation
def cole(f,e,D,Rep):
    resid=(1/f**0.5)+2*math.log10((e/(3.7*D))+2.51/(Rep*f**0.5))
    return resid

In [8]:
#Pressure Drop in Pipe
def Fhal(e,D,Rep):
    if Rep<=4000: #Haaland’s equation
        f=1/(-1.8*math.log10(((e/D)/3.7)**1.1+(6.9/Rep)))**2
    else:  #Colebrook–White equation
        f=fsolve(cole,0.01,args=(e,D,Rep),xtol=1e-10)
    return f

In [9]:
#Fanning friction factor
def Ffan(Rep):
    f=16/Rep
    return f

In [10]:
#Pressure drop in Frac
#P. A. WITHERSPOON #Validity of Cubic Law for Fluid Flow in a Deformable Rock Fracture 
def Pfrac(q,L,W,af,ff):
    Pfrac=12*8.91e-4*q*L*ff/(W*af**3)
    return Pfrac

In [11]:
#Flow rate in Fracs with Darcy and all Perforations
def temp1(Qfrac,Pfrac,L,A,k,Cd,Np,DPinj,DPpro):
    resid=Pfrac-(Qfrac*L*8.91e-4/(k*A))-(Qfrac**2*1000*0.808060652/((Cd**2)*(Np**2)*(DPinj**4)))-(Qfrac**2*1000*0.808060652/((Cd**2)*(Np**2)*(DPpro**4)))
    return resid
def Q_all_perfs(Pfrac,L,A,k,Cd,Np,DPinj,DPpro):
    Qfrac=fsolve(temp1,0.01,args=(Pfrac,L,A,k,Cd,Np,DPinj,DPpro),xtol=1e-10)
    return Qfrac

In [12]:
#Flow rate in Fracs with Darcy and only Injection Perforations
def temp2(Qfrac,Pfrac,L,A,k,Cd,Np,DPinj):
    resid=Pfrac-(Qfrac*L*8.91e-4/(k*A))-(Qfrac**2*1000*0.808060652/((Cd**2)*(Np**2)*(DPinj**4)))
    return resid
def Q_Injection_perfs(Pfrac,L,A,k,Cd,Np,DPinj):
    Qfrac=fsolve(temp2,0.01,args=(Pfrac,L,A,k,Cd,Np,DPinj),xtol=1e-10)
    return Qfrac

In [13]:
#Flow rate in Fracs with Darcy and only Production Perforations
def temp3(Qfrac,Pfrac,L,A,k,Cd,Np,DPpro):
    resid=Pfrac-(Qfrac*L*8.91e-4/(k*A))-(Qfrac**2*1000*0.808060652/((Cd**2)*(Np**2)*(DPpro**4)))
    return resid
def Q_Production_perfs(Pfrac,L,A,k,Cd,Np,DPpro):
    Qfrac=fsolve(temp3,0.01,args=(Pfrac,L,A,k,Cd,Np,DPpro),xtol=1e-10)
    return Qfrac

In [14]:
#Flow rate in Fracs with Darcy and no Perforations
def temp4(Qfrac,Pfrac,L,A,k):
    resid=Pfrac-(Qfrac*L*8.91e-4/(k*A))-(Qfrac**2*1000*0.808060652)-(Qfrac**2*1000*0.808060652)
    return resid
def Q_No_perfs(Pfrac,L,A,k):
    Qfrac=fsolve(temp4,0.01,args=(Pfrac,L,A,k),xtol=1e-10)
    return Qfrac

In [15]:
#Pressure drop in Frac Darcy
#Darcy's Law
def Pfdarcy(q,L,A,k):
    Pfrac=q*8.91e-4*L/(k*A)
    return Pfrac

In [16]:
#Flow rate in Fracs Darcy
#Darcy's Law
def Qfdarcy(Pfrac,L,A,k):
    q=Pfrac*k*A/(L*8.91e-4)
    return q

In [17]:
#Pressure drop in Perforations
#Weddle and Cramer
def Pperf(q,Cd,Np,Dp):
    Pperf=0.808060652*(q**2)*1000/((Cd**2)*(Np**2)*(Dp**4))
    return Pperf

In [18]:
#Pressure plots
def pplots(P,n,name,save,path):
    distpct=np.empty(n)
    temp=np.empty(n)
    plt.figure(figsize = (8,5))
    pmin=min(P)
    pmax=max(P)
    plt.ylim(ymin=pmin*0.95)
    plt.ylim(ymax=pmax*1.05)
    if name=="Width":
        plt.ylim(ymin=pmin*0.90*1000)
        plt.ylim(ymax=pmax*1.1*1000)
    # set width of bar
    barWidth = 1*(n-1)/(n+1)

    for i in range(0,n):
        distpct[i]=P[i]
        if name=="Width":
            distpct[i]=P[i]*1000
    x = np.linspace(1, n, n)
    plt.bar(x,distpct,width=barWidth,zorder=2,color='teal')
    # Add xticks on the middle of the group bars
    plt.xticks(np.arange(1, n+1, step=1))
    plt.xlabel('Fracture Number', fontweight='bold',fontsize=14)
    plt.ylabel('Pressure (Pa)', fontweight='bold',fontsize=14)
    plt.title('\n%s Pressure Distribution in Fractures' %name, fontweight='bold',fontsize=18)
    if name=="Permeability":
        plt.yscale('log')
        plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
        plt.title('\n%s Distribution in Fractures' %name, fontweight='bold',fontsize=18)
        plt.ylabel('Permeability m2', fontweight='bold',fontsize=14)
        plt.hlines(math.prod(P)**(1/len(P)), 0, n+1, colors='r', linestyles='solid',zorder=3)  
    #plt.legend(loc="upper center",ncol=3,bbox_to_anchor= (0.5, -0.15),prop={'size': 12},frameon=True)
    plt.grid(which='major', axis='both',color='lightgrey',zorder=1)
    if save=="yes":
        plt.savefig(r'%s/%s.png' %(path,name), bbox_inches='tight')
    return plt.show()
    